# BEE 4750 Homework 5: Mixed Integer and Stochastic Programming

**Name**: Camden Wehrle, Sarah Bae, Christine Berlingeri

**ID**: cw566, ssb238, cnb53

> **Due Date**
>
> Thursday, 12/05/24, 9:00pm

## Overview

### Instructions

-   In Problem 1, you will use mixed integer programming to solve a
    waste load allocation problem.

### Load Environment

The following code loads the environment and makes sure all needed
packages are installed. This should be at the start of most Julia
scripts.

In [7]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()
Pkg.add("JuMP")
Pkg.add("HiGHS")
Pkg.add("GraphRecipes")
Pkg.add("DataFrames")
Pkg.add("Plots")
Pkg.add("Measures")
Pkg.add("MarkdownTables")

  Activating project at `c:\Users\13158\Downloads`
   Resolving package versions...
  No Changes to `C:\Users\13158\Downloads\Project.toml`
  No Changes to `C:\Users\13158\Downloads\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\13158\Downloads\Project.toml`
  No Changes to `C:\Users\13158\Downloads\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\13158\Downloads\Project.toml`
  No Changes to `C:\Users\13158\Downloads\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\13158\Downloads\Project.toml`
  No Changes to `C:\Users\13158\Downloads\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\13158\Downloads\Project.toml`
  No Changes to `C:\Users\13158\Downloads\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\13158\Downloads\Project.toml`
  No Changes to `C:\Users\13158\Downloads\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\13158\Download

In [8]:
using JuMP
using HiGHS
using DataFrames
using GraphRecipes
using Plots
using Measures
using MarkdownTables

## Problems (Total: 30 Points)

### Problem 1 (30 points)

Three cities are developing a coordinated municipal solid waste (MSW)
disposal plan. Three disposal alternatives are being considered: a
landfill (LF), a materials recycling facility (MRF), and a
waste-to-energy facility (WTE). The capacities of these facilities and
the fees for operation and disposal are provided below.

-   **LF**: Capacity 200 Mg, fixed cost \$2000/day, tipping cost
    \$50/Mg;
-   **MRF**: Capacity 350 Mg, fixed cost \$1500/day, tipping cost
    \$7/Mg, recycling cost \$40/Mg recycled;
-   **WTE**: Capacity 210 Mg, fixed cost \$2500/day, tipping cost
    \$60/Mg;

The MRF recycling rate is 40%, and the ash fraction of non-recycled
waste is 16% and of recycled waste is 14%. Transportation costs are
\$1.5/Mg-km, and the relative distances between the cities and
facilities are provided in the table below.

| **City/Facility** | **Landfill (km)** | **MRF (km)** | **WTE (km)** |
|:-----------------:|:-----------------:|:------------:|:------------:|
|         1         |         5         |      30      |      15      |
|         2         |        15         |      25      |      10      |
|         3         |        13         |      45      |      20      |
|        LF         |        \-         |      32      |      18      |
|        MRF        |        32         |      \-      |      15      |
|        WTE        |        18         |      15      |      \-      |

The fixed costs associated with the disposal options are incurred only
if the particular disposal option is implemented. The three cities
produce 100, 90, and 120 Mg/day of solid waste, respectively, with the
composition provided in the table below.

**Reminder**: Use `round(x; digits=n)` to report values to the
appropriate precision!

**In this problem**:

-   Formulate the waste load allocation problem and implement it in
    `JuMP`.
-   Draw a diagram showing the flows of waste between the cities and the
    facilities. Which facilities (if any) will not be used? Does this
    solution make sense?




**PROBLEM SOLUTION BEGINS**

First, we looked at the variables of the network. This allowed us to 
visualize how waste was flowing through the whole network of cities 
and facilities.

| **City/Facility** | **Waste Produced (Mg/day)** | **Capacity of Disposal (Mg/day)** |
|:-----------------:|:-----------------:|:------------:
|         1         |         100         |      \-      | 
|         2         |        90         |      \-      | 
|         3         |        120         |     \-      | 
|        LF         |        \-         |      200      | 
|        MRF        |        \-         |      350     | 
|        WTE        |        \-         |      210      |

The decision variables are the amount of waste transported from each city to each disposal facility,
the residual waste transported from each disposal to each other disposal, and whether or not each
facility is "on" (since we have the option to not use one or more of the facilities).

There should be 9 waste variables as there are nine potential components
to the waste disposal. Specifically, these are

$W_1$ = Waste amount from city 1 to WTE

$W_2$ = Waste amount from city 2 to WTE

$W_3$ = Waste amount from city 3 to WTE

$W_4$ = Waste amount from city 1 to MRF

$W_5$ = Waste amount from city 2 to MRF

$W_6$ = Waste amount from city 3 to MRF

$W_7$ = Waste amount from city 1 to LF

$W_8$ = Waste amount from city 2 to LF

$W_9$ = Waste amount from city 3 to LF

For the residual waste, the possible scenarios are

$R_1$ = Residual from MRF to WTE

$R_2$ = Residual from WTE to LF

$R_3$ = Residual from MRF to LF

In addition, we need to have variables to represent the binary on/off status of each facility:

$Y_1$ = Operational status of WTE

$Y_2$ = Operational status of MRF

$Y_3$ = Operational status of LF

The overall objective of this problem is to minimize the cost of waste disposal (both from actual
disposal costs and transportation costs).

The transportation costs will be represented by an equation including the following:

$$TranspoCosts = 1.5*(Distance *WasteAmounts)$$ 
for each combination of travel from each city/facility.

The disposal costs will be represented by the following:

$$FixedCost*OpStatus + Tipping*(WasteAmounts + ResidualWaste)$$
for each facility

Combining these two equations using the previously defined variables and given values in the
problem statement, our total cost equation and objective statement is represented by:

$$2500Y_1 + 1500Y_2 + 2000Y_3 + 76.5W_1 + 75W_2 
+ 90W_3 + 68W_4 + 60.5W_5 + 67.5W_6 + 57.5W_7 + 72.5W_8 + 69.5W_9
+ 82.5R_1 + 77R_2 + 98R_3$$

Constraints for this problem are that each facility has a treatment capacity that cannot be 
exceeded, and all of the waste produced by each city in a day must be disposed of. Also,
we know what fraction of residual waste will go to each facility. 

Constraint on needing to dispose of all waste:
$$W_1 + W_4 + W_5 = 100$$ 
for City 1
$$W_2 + W_5 + W_8 = 90$$ 
for City 2
$$W_3 + W_6 + W_9 = 90$$ 
for City 3

Constraint on facility disposal limits:

$$W_1 + W_2 + W_3 + R_1 <= 210$$
for WTE
$$W_4 + W_5 + W_6 <= 350$$
for MRF
$$W_7 + W_8 + W_9 + R_2 + R_3 <= 200$$
for LF

Constraint on amount of residual waste:

$$R_1 + R_3 = .54(W_4 + W_5 + W_)$$
$$R_2 = .16(W_1 + W_2 + W_3 + R_1)$$

Finally, we will use the binary indicators to see if each facility 
will be used or not. We will use the big "M", set to an arbitrarily
large value.

$$M*Y_1 >= W_1 + W_2 + W_3$$
for WTE
$$M*Y_2 >= W_4 + W_5 + W_6$$
for MRF
$$M*Y_3 >= W_7 + W_8 + W_9$$
for LF




In [21]:
waste_op = Model(HiGHS.Optimizer)
@variable(waste_op, W[1:9] >= 0)
@variable(waste_op, R[1:3] >=0)
@variable(waste_op, Y[1:3], Bin)

M = 10000000

#Write objective (minimize cost of transportation and disposal)
@objective(waste_op, Min, 2500*Y[1] + 1500*Y[2] + 2000*Y[3] + 82.5*W[1] + 75*W[2] 
+ 90*W[3] + 68*W[4] + 60.5*W[5] + 67.5*W[6] + 57.5*W[7] + 72.5*W[8] + 69.5*W[9]
+ 82.5*R[1] + 77*R[2] + 98*R[3])

#Constraint: all waste from each city must be disposed of
@constraint(waste_op, W[1] + W[4] + W[7] == 100)
@constraint(waste_op, W[2] + W[5] + W[8] == 90)
@constraint(waste_op, W[3] + W[6] + W[9] == 120)

#Constraint: Cannot send more waste to a facility than 
#the facility can handle
@constraint(waste_op, W[1] + W[2] + W[3] + R[1] <= 210)
@constraint(waste_op, W[4] + W[5] + W[6] <= 350)
@constraint(waste_op, W[7] + W[8] + W[9] + R[2] + R[3] <= 200)

#Constraint: Limits on amount of residual waste produced
@constraint(waste_op, R[1] + R[3] == .54*(W[4] + W[5] + W[6]))
@constraint(waste_op, R[2] == .16*(W[1] + W[2] + W[3] + R[1]))

#Constraint: See if each facility should be used or not
@constraint(waste_op, M*Y[1] >= W[1] + W[2] + W[3])
@constraint(waste_op, M*Y[2] >= W[4] + W[5] + W[6])
@constraint(waste_op, M*Y[3] >= W[7] + W[8] + W[9])

optimize!(waste_op)
@show value.(W);
@show value.(Y);
@show value.(R);
@show objective_value(waste_op);

Running HiGHS 1.7.2 (git hash: 5ce7a2753): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [2e-01, 1e+07]
  Cost   [6e+01, 2e+03]
  Bound  [1e+00, 1e+00]
  RHS    [9e+01, 4e+02]
Presolving model
10 rows, 15 cols, 40 nonzeros  0s
9 rows, 14 cols, 38 nonzeros  0s

Solving MIP model with:
   9 rows
   14 cols (3 binary, 0 integer, 0 implied int., 11 continuous)
   38 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   0               inf                  inf        0      0      0         0     0.0s
 S       0       0         0   0.00%   0               27792.857143     100.00%        0      0      0         0     0.0s
         0       0         0   0.00%   25504.070661    27792.857143       8.24%        0      0      0  

As can be seen from the model results above, the WTE and LF facilites are used, but the MRF is not used. This makes sense
because based on the variable costs, the recycling rate is high, and a significant portion (40%) of the waste from that 
facility is recycled. This also means that a lot of the waste has to be sent on to further facilities, making this an 
expensive option. It is also far from all the cities. 

Overall, the optimal solution will result in a daily cost of $27,792. 90 kg of waste will be sent from city 2 to WTE,
40.95 from city 3 to WTE, 100 from city 1 to LF, and 79.05 from city 3 to LF. 20.95 kg of residual will be sent 
from WTE to LF. It makes sense that there's no other residual waste flows since we are not using MRF. See the below 
diagram to visualize the flow of waste from cities and facilites.

## References

List any external references consulted, including classmates.